In [1]:
from IPython.display import display
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.decomposition import PCA
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
original_data = pd.read_csv("train.csv")

train = original_data

# print(train.isnull().sum())

train.describe()
# train.head()






,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,26729,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,26729,6374,22918,5,16,2,5,44,1380,366
top,A671302,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,1,136,19,10769,7816,15595,9779,3969,8810,2824


# Age Feature Transform

In [3]:
# Preprocess the AgeuponOutcome Data
age_arr = train['AgeuponOutcome']

num_year = 0
num_month = 0
num_week = 0
num_day = 0
num_else = 0
num_nan = 0

year_index = []
month_index = []
week_index = []
day_index = []
else_index = []
nan_index = []

for i, age in enumerate(age_arr):
    if type(age) == type('string'):
        if age.find('year') != -1:
            year_index.append(i)
            num_year += 1
        elif 'month' in age:
            month_index.append(i)
            num_month += 1
        elif 'week' in age:
            week_index.append(i)
            num_week += 1
        elif 'day' in age:
            day_index.append(i)
            num_day += 1
        else:
            else_index.append(i)
            num_else += 1
    else:
        nan_index.append(i)
        num_nan += 1
        
# Transform the year values to float
for i in year_index:
    age_arr[i] = float(age_arr[i].split(' ')[0])
#     print(age_arr[i])

# Transform the month values to float
for i in month_index:
    age_arr[i] = float(age_arr[i].split(' ')[0]) / 12
#     print(age_arr[i])
    
# Transform the week values to float
for i in week_index:
    age_arr[i] = float(age_arr[i].split(' ')[0]) / 52
#     print(age_arr[i])

# Transform the day values to float
for i in day_index:
    age_arr[i] = float(age_arr[i].split(' ')[0]) / 365
#     print(age_arr[i])
    
age_mean = np.mean([x for x in age_arr if np.isfinite(x)])

age_arr = [age_mean if np.isnan(x) else x for x in age_arr]

train['age_float'] = age_arr

display(train.describe())

# imr = Imputer(strategy='mean', axis=0)
# imr.fit(train['age_float'].reshape(-1, 1))

# # NEED TO CHECK THIS PROCESS LATER
# imputed_age = imr.transform(train['age_float'].reshape(-1, 1))
# imputed_age = [y 
#                for x in imputed_age 
#                for y in x]

# train = train.drop('age_float', 1)
# train['age_float'] = imputed_age[0]
train.describe(include='all')


,age_float
count,26729.000000
mean,2.177244
std,2.964768
min,0.000000
25%,0.166667
50%,1.000000
75%,3.000000
max,20.000000


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,age_float
count,26729,19038,26729,26729,13117,26729,26728,26711.0,26729,26729,26729.000000
unique,26729,6374,22918,5,16,2,5,43.0,1380,366,NaN
top,A671302,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1.0,Domestic Shorthair Mix,Black/White,NaN
freq,1,136,19,10769,7816,15595,9779,3969.0,8810,2824,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.177244
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.964768
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000


# Constructing Color Columns

In [4]:
color_arr = train['Color']

# Pure Color
pure_yes = []
pure_no = []

for i, color in enumerate(color_arr):
    if '/' in color:
        pure_no.append(i)
    else:
        pure_yes.append(i)

pure_arr = [None] * (len(pure_yes) + len(pure_no))

for i in pure_yes:
    pure_arr[i] = 'Yes'
for i in pure_no:
    pure_arr[i] = 'No'

train['pure_color'] = pure_arr


# Black Color
pure_dict = dict(train['Color'].ix[pure_yes])

black_arr = ['No'] * train.shape[0]

for i, color in pure_dict.items():
    if 'black' in color.lower():
        black_arr[i] = 'Yes'
    else:
        black_arr[i] = 'No'

train['black_color'] = black_arr

train.describe(include='all')

# Tabby Color
tabby_arr = ['No'] * train.shape[0]

for i, color in pure_dict.items():
    if 'tabby' in color.lower():
        tabby_arr[i] = 'Yes'
    else:
        tabby_arr[i] = 'No'

train['tabby_color'] = tabby_arr

train.describe(include='all')

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,age_float,pure_color,black_color,tabby_color
count,26729,19038,26729,26729,13117,26729,26728,26711.0,26729,26729,26729.000000,26729,26729,26729
unique,26729,6374,22918,5,16,2,5,43.0,1380,366,NaN,2,2,2
top,A671302,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1.0,Domestic Shorthair Mix,Black/White,NaN,No,No,No
freq,1,136,19,10769,7816,15595,9779,3969.0,8810,2824,NaN,13924,24329,23517
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.177244,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.964768,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN


# Dog Breed Transform

In [5]:
# Load the Dog Dictionary
dogs = pd.read_csv("dog_dictionary.csv");
# print(dogs.head());
dog_dict = dogs.set_index('Dog Breed')['Size'].to_dict()

# Create Size Column
breed_arr = train['Breed'].values
type_list = dog_dict.keys()
size_arr = ['NaN'] * train.shape[0]

for id, breed in enumerate(breed_arr):
    for tp in type_list:
        if (tp in breed):
            size_arr[id] = dog_dict[tp]

train['size'] = size_arr

# print(train[train['size'] == 'NaN'])
# otherDog = train.loc[(train['AnimalType'] == 'Dog') & (train['size'] == 'NaN')]
# print(otherDog.describe(include='all'))
# otherDog.to_csv('otherDog.csv')

# Create the mix column
mix_breed = []
pure_breed = []

for i, breed in enumerate(breed_arr):
    if 'mix' in breed.lower():
        mix_breed.append(i)
    else:
        pure_breed.append(i)
        
mix_arr = ['Yes'] * train.shape[0]

for i in pure_breed:
    mix_arr[i] = 'No'

train['mix'] = mix_arr

train.describe(include='all')


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,age_float,pure_color,black_color,tabby_color,size,mix
count,26729,19038,26729,26729,13117,26729,26728,26711.0,26729,26729,26729.000000,26729,26729,26729,26729,26729
unique,26729,6374,22918,5,16,2,5,43.0,1380,366,NaN,2,2,2,6,2
top,A671302,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1.0,Domestic Shorthair Mix,Black/White,NaN,No,No,No,NaN,Yes
freq,1,136,19,10769,7816,15595,9779,3969.0,8810,2824,NaN,13924,24329,23517,11136,22298
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.177244,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.964768,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN


# Cat Breed Transform

In [6]:
# Load the Cat Dictionary
cats = pd.read_csv("cat_dictionary.csv");
# print(dogs.head());
cat_dict = cats.set_index('Cat Breed')['Longhair'].to_dict()

# Create Longhair Column
breed_arr_cat = train['Breed'].values
type_list_cat = cat_dict.keys()
longhair_arr = ['NaN'] * train.shape[0]

# Match with the cat_dict
for id, breed in enumerate(breed_arr_cat):
    for tp in type_list_cat:
        if (tp in breed):
            longhair_arr[id] = cat_dict[tp]
train['Longhair'] = longhair_arr

# Filter out the Shorthair
for id, breed in enumerate(breed_arr_cat):
    if ('shorthair' in breed.lower()):
        longhair_arr[id] = 'No'

# Filter out the 'Medium Hair'
for id, breed in enumerate(breed_arr_cat):
    if ('Medium Hair' in breed):
        longhair_arr[id] = 'No'
        
train['Longhair'] = longhair_arr
# otherCat = train.loc[(train['AnimalType'] == 'Cat') & (train['Longhair'] == 'NaN')]
# print(otherCat.describe(include='all'))
# otherCat.to_csv('otherCat.csv')

# Create Medium Hair column
medium_hair_arr = ['No'] * train.shape[0]

for id, breed in enumerate(breed_arr_cat):
    if ('Medium Hair' in breed):
        medium_hair_arr[id] = 'Yes'

train['MediumHair'] = medium_hair_arr

# allCat = train.loc[(train['AnimalType'] == 'Cat')]
# print(allCat.describe(include='all'))

# Assign 'NaN' to Dogs got assigned 'No' to 'Longhair' and 'MediumHair'
train.loc[(train['AnimalType'] == 'Dog') & (train['Longhair'] == 'No'), 'Longhair'] = 'NaN'
train.loc[(train['AnimalType'] == 'Dog') & (train['MediumHair'] == 'No'), 'MediumHair'] = 'NaN'

train.describe(include='all')

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,age_float,pure_color,black_color,tabby_color,size,mix,Longhair,MediumHair
count,26729,19038,26729,26729,13117,26729,26728,26711.0,26729,26729,26729.000000,26729,26729,26729,26729,26729,26729,26729
unique,26729,6374,22918,5,16,2,5,43.0,1380,366,NaN,2,2,2,6,2,3,3
top,A671302,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1.0,Domestic Shorthair Mix,Black/White,NaN,No,No,No,NaN,Yes,NaN,NaN
freq,1,136,19,10769,7816,15595,9779,3969.0,8810,2824,NaN,13924,24329,23517,11136,22298,15129,15595
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.177244,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.964768,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# DateTime Transform

In [7]:
from dateutil.parser import parse
datetime_arr = train['DateTime']

#Create Weekday, Hour columns
weekday_arr = [x.weekday() for x in map(parse, datetime_arr)]
hour_arr = [x.hour for x in map(parse, datetime_arr)]

train['Weekday'] = weekday_arr
train['Hour'] = hour_arr

print(train.describe(include='all'))

       AnimalID   Name             DateTime OutcomeType OutcomeSubtype  \
count     26729  19038                26729       26729          13117   
unique    26729   6374                22918           5             16   
top     A671302    Max  2015-08-11 00:00:00    Adoption        Partner   
freq          1    136                   19       10769           7816   
mean        NaN    NaN                  NaN         NaN            NaN   
std         NaN    NaN                  NaN         NaN            NaN   
min         NaN    NaN                  NaN         NaN            NaN   
25%         NaN    NaN                  NaN         NaN            NaN   
50%         NaN    NaN                  NaN         NaN            NaN   
75%         NaN    NaN                  NaN         NaN            NaN   
max         NaN    NaN                  NaN         NaN            NaN   

       AnimalType SexuponOutcome  AgeuponOutcome                   Breed  \
count       26729          26728   


# Named and the Nonames

In [8]:
name_arr = train['Name']
have_name_arr = ['No'] * train.shape[0]

for id, name in enumerate(name_arr):
    if type(name) != type('string'):
        if np.isnan(name):
            have_name_arr[id] = 'No'
    else:
        have_name_arr[id] = 'Yes'

train['HaveName'] = have_name_arr
# print(train.describe(include='all'))

train['OutcomeSubtype'].isnull().sum()

13612

# Split Sex and Intactness

In [9]:
sex_intact_arr = train['SexuponOutcome']

split_arr = [x.split(' ') if type(x) == type('string') else x for x in sex_intact_arr]

sex_arr = [x[-1] if type(x) == type([1]) else x for x in split_arr]
intact_arr = [x[0] if type(x) == type([1]) else x for x in split_arr]

train['Sex'] = sex_arr
train['Intactness'] = intact_arr

# Fill the Missing Values with Mode
train.loc[(train['Sex'].isnull()), 'Sex'] = Counter(sex_arr).most_common(1)[0][0]
train.loc[(train['Intactness'].isnull()), 'Intactness'] = Counter(intact_arr).most_common(1)[0][0]

print(train.describe(include='all'))

# print(Counter(sex_arr).keys())
# print(Counter(intact_arr).keys())


    

       AnimalID   Name             DateTime OutcomeType OutcomeSubtype  \
count     26729  19038                26729       26729          13117   
unique    26729   6374                22918           5             16   
top     A671302    Max  2015-08-11 00:00:00    Adoption        Partner   
freq          1    136                   19       10769           7816   
mean        NaN    NaN                  NaN         NaN            NaN   
std         NaN    NaN                  NaN         NaN            NaN   
min         NaN    NaN                  NaN         NaN            NaN   
25%         NaN    NaN                  NaN         NaN            NaN   
50%         NaN    NaN                  NaN         NaN            NaN   
75%         NaN    NaN                  NaN         NaN            NaN   
max         NaN    NaN                  NaN         NaN            NaN   

       AnimalType SexuponOutcome  AgeuponOutcome                   Breed  \
count       26729          26728   

# Drop Useless Columns

In [10]:
# Drop AnimalID, Name, DateTime, OutcomeSubtype, SexuponOutcome, AgeuponOutcome, Breed, Color
train.drop(train.columns[[0, 1, 2, 4, 6, 7, 8, 9]], axis=1, inplace=True)

train.describe(include='all')

,OutcomeType,AnimalType,age_float,pure_color,black_color,tabby_color,size,mix,Longhair,MediumHair,Weekday,Hour,HaveName,Sex,Intactness
count,26729,26729,26729.000000,26729,26729,26729,26729,26729,26729,26729,26729.000000,26729.000000,26729,26729,26729
unique,5,2,NaN,2,2,2,6,2,3,3,NaN,NaN,2,3,4
top,Adoption,Dog,NaN,No,No,No,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Male,Neutered
freq,10769,15595,NaN,13924,24329,23517,11136,22298,15129,15595,NaN,NaN,19038,13305,9780
mean,NaN,NaN,2.177244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.106364,14.445396,NaN,NaN,NaN
std,NaN,NaN,2.964768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.046001,3.337331,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,12.000000,NaN,NaN,NaN
50%,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,15.000000,NaN,NaN,NaN
75%,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,17.000000,NaN,NaN,NaN


# Label Transform

In [11]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(train['OutcomeType']))}
inv_class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
print(inv_class_mapping)
train['OutcomeType'] = train['OutcomeType'].map(class_mapping)
train.head()


{'Transfer': 4, 'Euthanasia': 2, 'Died': 1, 'Return_to_owner': 3, 'Adoption': 0}
{0: 'Adoption', 1: 'Died', 2: 'Euthanasia', 3: 'Return_to_owner', 4: 'Transfer'}


,OutcomeType,AnimalType,age_float,pure_color,black_color,tabby_color,size,mix,Longhair,MediumHair,Weekday,Hour,HaveName,Sex,Intactness
0,3,Dog,1.000000,No,No,No,Medium-Small,Yes,NaN,NaN,2,18,Yes,Male,Neutered
1,2,Cat,1.000000,Yes,No,Yes,NaN,Yes,No,No,6,12,Yes,Female,Spayed
2,0,Dog,2.000000,No,No,No,Medium-Small,Yes,NaN,NaN,5,12,Yes,Male,Neutered
3,4,Cat,0.057692,Yes,No,No,NaN,Yes,No,No,4,19,No,Male,Intact
4,4,Dog,2.000000,Yes,No,No,Small,No,NaN,NaN,4,12,No,Male,Neutered


# Ordinal Features Transform

In [12]:
size_mapping = {'NaN': 0, 'Small': 1, 'Medium-Small': 2, 'Medium': 3, 'Medium-Large': 4, 'Large': 5}
inv_size_mapping = {v: k for k, v in size_mapping.items()}
print(size_mapping)
print(inv_size_mapping)
train['size'] = train['size'].map(size_mapping)
train.head()

{'Medium-Small': 2, 'NaN': 0, 'Medium': 3, 'Large': 5, 'Small': 1, 'Medium-Large': 4}
{0: 'NaN', 1: 'Small', 2: 'Medium-Small', 3: 'Medium', 4: 'Medium-Large', 5: 'Large'}


,OutcomeType,AnimalType,age_float,pure_color,black_color,tabby_color,size,mix,Longhair,MediumHair,Weekday,Hour,HaveName,Sex,Intactness
0,3,Dog,1.000000,No,No,No,2,Yes,NaN,NaN,2,18,Yes,Male,Neutered
1,2,Cat,1.000000,Yes,No,Yes,0,Yes,No,No,6,12,Yes,Female,Spayed
2,0,Dog,2.000000,No,No,No,2,Yes,NaN,NaN,5,12,Yes,Male,Neutered
3,4,Cat,0.057692,Yes,No,No,0,Yes,No,No,4,19,No,Male,Intact
4,4,Dog,2.000000,Yes,No,No,1,No,NaN,NaN,4,12,No,Male,Neutered


# Nominal Feature Transform

In [13]:
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(categorical_features = [5], sparse = False)

# Convert Weekday to String
weekday_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
inv_weekday_mapping = {v: k for k, v in weekday_mapping.items()}
print(weekday_mapping)
print(inv_weekday_mapping)
train['Weekday'] = train['Weekday'].map(weekday_mapping)
train.head()

train = pd.get_dummies(train)


{0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
{'Monday': 0, 'Friday': 4, 'Sunday': 6, 'Wednesday': 2, 'Thursday': 3, 'Tuesday': 1, 'Saturday': 5}


In [14]:
train.head()

,OutcomeType,age_float,size,Hour,AnimalType_Cat,AnimalType_Dog,pure_color_No,pure_color_Yes,black_color_No,black_color_Yes,...,Weekday_Wednesday,HaveName_No,HaveName_Yes,Sex_Female,Sex_Male,Sex_Unknown,Intactness_Intact,Intactness_Neutered,Intactness_Spayed,Intactness_Unknown
0,3,1.000000,2,18,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2,1.000000,0,12,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,2.000000,2,12,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,4,0.057692,0,19,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,4,2.000000,1,12,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [15]:
train.columns.values

array(['OutcomeType', 'age_float', 'size', 'Hour', 'AnimalType_Cat',
       'AnimalType_Dog', 'pure_color_No', 'pure_color_Yes',
       'black_color_No', 'black_color_Yes', 'tabby_color_No',
       'tabby_color_Yes', 'mix_No', 'mix_Yes', 'Longhair_NaN',
       'Longhair_No', 'Longhair_Yes', 'MediumHair_NaN', 'MediumHair_No',
       'MediumHair_Yes', 'Weekday_Friday', 'Weekday_Monday',
       'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday',
       'Weekday_Tuesday', 'Weekday_Wednesday', 'HaveName_No',
       'HaveName_Yes', 'Sex_Female', 'Sex_Male', 'Sex_Unknown',
       'Intactness_Intact', 'Intactness_Neutered', 'Intactness_Spayed',
       'Intactness_Unknown'], dtype=object)

# Prepare the Datasets
## Separate Dataset into Training and Testing sets

In [16]:
X = train.drop(train.columns[[0]], axis=1)
y = train['OutcomeType']
samples_indices = [1, 2, 1001]
samples_X = pd.DataFrame(X.loc[samples_indices], columns = X.keys()).reset_index(drop = True)
display(samples_X)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.25, random_state = 1)

,age_float,size,Hour,AnimalType_Cat,AnimalType_Dog,pure_color_No,pure_color_Yes,black_color_No,black_color_Yes,tabby_color_No,...,Weekday_Wednesday,HaveName_No,HaveName_Yes,Sex_Female,Sex_Male,Sex_Unknown,Intactness_Intact,Intactness_Neutered,Intactness_Spayed,Intactness_Unknown
0,1.0,0,12,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,2,12,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,2.0,5,18,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


# Feature Scaling

In [17]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

display(X_train.head())
display(pd.DataFrame(X_train_std).describe())

,age_float,size,Hour,AnimalType_Cat,AnimalType_Dog,pure_color_No,pure_color_Yes,black_color_No,black_color_Yes,tabby_color_No,...,Weekday_Wednesday,HaveName_No,HaveName_Yes,Sex_Female,Sex_Male,Sex_Unknown,Intactness_Intact,Intactness_Neutered,Intactness_Spayed,Intactness_Unknown
20430,3.000000,0,13,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2845,2.000000,1,17,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
16949,0.666667,4,14,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7860,8.000000,4,20,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
18423,2.000000,1,15,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
count,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,...,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04,2.004600e+04
mean,-3.899017e-18,1.736835e-17,-5.813080e-17,-8.684175e-17,-2.357133e-17,-1.807726e-17,-1.027923e-16,-7.089122e-19,2.109014e-17,9.357642e-17,...,3.260996e-17,5.494070e-17,-5.494070e-17,2.365995e-17,-2.835649e-18,-1.098814e-17,-3.863572e-17,-6.238428e-17,5.370010e-17,-1.098814e-17
std,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,...,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00,1.000025e+00
min,-7.307994e-01,-8.981078e-01,-4.358248e+00,-8.464125e-01,-1.181457e+00,-1.044175e+00,-9.576934e-01,-3.150278e+00,-3.174323e-01,-2.702602e+00,...,-3.865574e-01,-6.367532e-01,-1.570467e+00,-9.264707e-01,-9.943292e-01,-2.066514e-01,-5.982091e-01,-7.586645e-01,-7.021125e-01,-2.066514e-01
25%,-6.744345e-01,-8.981078e-01,-7.399572e-01,-8.464125e-01,-1.181457e+00,-1.044175e+00,-9.576934e-01,3.174323e-01,-3.174323e-01,3.700137e-01,...,-3.865574e-01,-6.367532e-01,-1.570467e+00,-9.264707e-01,-9.943292e-01,-2.066514e-01,-5.982091e-01,-7.586645e-01,-7.021125e-01,-2.066514e-01
50%,-3.926096e-01,-2.825835e-01,1.646154e-01,-8.464125e-01,8.464125e-01,9.576934e-01,-9.576934e-01,3.174323e-01,-3.174323e-01,3.700137e-01,...,-3.865574e-01,-6.367532e-01,6.367532e-01,-9.264707e-01,-9.943292e-01,-2.066514e-01,-5.982091e-01,-7.586645e-01,-7.021125e-01,-2.066514e-01
75%,2.837700e-01,9.484649e-01,7.676638e-01,1.181457e+00,8.464125e-01,9.576934e-01,1.044175e+00,3.174323e-01,-3.174323e-01,3.700137e-01,...,-3.865574e-01,1.570467e+00,6.367532e-01,1.079365e+00,1.005703e+00,-2.066514e-01,1.671656e+00,1.318106e+00,1.424273e+00,-2.066514e-01
max,5.694807e+00,2.179513e+00,2.576809e+00,1.181457e+00,8.464125e-01,9.576934e-01,1.044175e+00,3.174323e-01,3.150278e+00,3.700137e-01,...,2.586938e+00,1.570467e+00,6.367532e-01,1.079365e+00,1.005703e+00,4.839067e+00,1.671656e+00,1.318106e+00,1.424273e+00,4.839067e+00


## Quick and Dirty Try
## SVM

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.metrics import make_scorer
from sklearn import grid_search
from sklearn.metrics import f1_score
import time

# TODO: Create a decision tree regressor and fit it to the training set
clf = svm.SVC(C = 100000, probability = True)
# parameters = {"C": [1, 100, 1000, 10000, 100000], "kernel": ['linear', 'poly', 'rbf'], "degree": [3, 4, 5], 'probability': [True]}
# parameters = {"estimator__C": [100000], "estimator__kernel": ['rbf'], "estimator__degree": [3]}

mclf = OneVsRestClassifier(clf)

# Cross-Validation + Grid Search
# svc_gs = grid_search.GridSearchCV(mclf, parameters, cv=9)

print("Training {}...".format(mclf.__class__.__name__))
train_start = time.time()
# svc_gs.fit(X_train, y_train)

mclf.fit(X_train, y_train)

train_end = time.time()
print("Done!\nTraining time (secs): {:.3f}".format(train_end - train_start))

y_pred = mclf.predict(X_train)
# y_pred = svc_gs.predict(X_train)

# TODO: Report the score of the prediction using the testing set
score = mclf.score(X_test, y_test)
print("R^2: {}".format(score))

print(y_pred)
print(mclf.predict_proba(X_train))
print(mclf)

# print(svc_gs.predict_proba(X_train))
# print(svc_gs.best_params_)
# print(svc_gs)


Training OneVsRestClassifier...


## Plot Learning Curve
### Define Learning Curve Function

In [ ]:
import matplotlib.pyplot as plt
from sklearn import cross_validation 
from sklearn.learning_curve import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curves (SVC)"
plot_learning_curve(mclf, title, X_train, y_train, cv=9, n_jobs=1, ylim=(0, 1), train_sizes=np.linspace(.1, 1.0, 10))

plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# TODO: Create a decision tree regressor and fit it to the training set
clf = RandomForestClassifier(n_estimators=1500)

mclf = OneVsRestClassifier(clf)
mclf.fit(X_train, y_train).predict(X_train)

# TODO: Report the score of the prediction using the testing set
score = mclf.score(X_test, y_test)
print("R^2: {}".format(score))

print(mclf.fit(X_train, y_train).predict(X_train))

In [ ]:
print(clf.feature_importances_)

# Apply PCA
In this section we use PCA to analyze the predicting power of each feature.

### Define PCA Plot Function

In [ ]:
def pca_results(good_data, pca):
    '''
    Create a DataFrame of the PCA results
    Includes dimension feature weights and explained variance
    Visualizes the PCA results
    '''

    # Dimension indexing
    dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = good_data.keys())
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    fig, ax = plt.subplots(figsize = (14,8))

    # Plot the feature weights as a function of the components
    components.plot(ax = ax, kind = 'bar');
    ax.set_ylabel("Feature Weights")
    ax.set_xticklabels(dimensions, rotation=0)


    # Display the explained variance ratios
    for i, ev in enumerate(pca.explained_variance_ratio_):
        ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

    # Return a concatenated DataFrame
    return pd.concat([variance_ratios, components], axis = 1)

## Train and Plot PCA 

In [ ]:
pca = PCA(n_components = 4)
pca.fit(X)
results = pca_results(X, pca)

## Transform Features into the PCA Space

In [ ]:
pca = PCA(n_components = 2)
pca.fit(X)
reduced_X = pca.transform(X)
pca_samples = pca.transform(samples_X)
reduced_X = pd.DataFrame(reduced_X, columns = ['Dimension 1', 'Dimension 2'])
display(reduced_X.describe())
display(pd.DataFrame(np.round(pca_samples, 2), columns = ['Dimension 1', 'Dimension 2']))

## Use K-Means to Insepct Structure of the Data

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import operator

# TODO: Apply your clustering algorithm of choice to the reduced data
scores = {}

# Grid Search for best K
# for n in range(2, 8):
#     clusterer = KMeans(n_clusters = n)
#     clusterer.fit(reduced_X)

#     # TODO: Predict the cluster for each data point
#     preds = clusterer.predict(reduced_X)

#     # TODO: Find the cluster centers
#     centers = clusterer.cluster_centers_

#     # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
#     labels = clusterer.labels_
#     score = silhouette_score(reduced_X, labels, metric='euclidean')
#     scores[n] = score
    
#     print("Silhouette score with n = {}: {}".format(n, score))

# max_key = max(scores.keys(), key=(lambda k: scores[k])) 

max_key = 3

clusterer = KMeans(n_clusters = max_key)
clusterer.fit(reduced_X)

# TODO: Predict the cluster for each data point
preds = clusterer.predict(reduced_X)

# TODO: Find the cluster centers
centers = clusterer.cluster_centers_

# TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
labels = clusterer.labels_
score = silhouette_score(reduced_X, labels, metric='euclidean')
        
print("Cluster centers: \n{}\n".format(centers))
print("Silhouette score (max) with n = {}: {}".format(max_key, score))

### Define Cluster Plot Function

In [ ]:
def cluster_results(reduced_data, preds, centers, pca_samples):
    '''
    Visualizes the PCA-reduced cluster data in two dimensions
    Adds cues for cluster centers and student-selected sample data
    '''

    predictions = pd.DataFrame(preds, columns = ['Cluster'])
    plot_data = pd.concat([predictions, reduced_data], axis = 1)

    # Generate the cluster plot
    fig, ax = plt.subplots(figsize = (14,8))

    # Color map
    cmap = cm.get_cmap('gist_rainbow')

    # Color the points based on assigned cluster
    for i, cluster in plot_data.groupby('Cluster'):   
        cluster.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', \
                     color = cmap((i)*1.0/(len(centers)-1)), label = 'Cluster %i'%(i), s=30);

    # Plot centers with indicators
    for i, c in enumerate(centers):
        ax.scatter(x = c[0], y = c[1], color = 'white', edgecolors = 'black', \
                   alpha = 1, linewidth = 2, marker = 'o', s=200);
        ax.scatter(x = c[0], y = c[1], marker='$%d$'%(i), alpha = 1, s=100);

    # Plot transformed sample points 
    ax.scatter(x = pca_samples[:,0], y = pca_samples[:,1], \
               s = 150, linewidth = 4, color = 'black', marker = 'x');

    # Set plot title
    ax.set_title("Cluster Learning on PCA-Reduced Data - Centroids Marked by Number\nTransformed Sample Data Marked by Black Cross");    

In [ ]:
cluster_results(reduced_X, preds, centers, pca_samples)

# Hierarchical Clustering

In [ ]:
# from sklearn.cluster import AgglomerativeClustering

# # TODO: Apply your clustering algorithm of choice to the reduced data
# scores_A = {}

# for n in range(2, 8):
#     clstr_A = AgglomerativeClustering(n_clusters = n)
#     clstr_A.fit(reduced_X)

#     # TODO: Predict the cluster for each data point
#     preds_A = clstr_A.predict(reduced_X)

#     # TODO: Find the cluster centers
#     centers_A = clstr_A.cluster_centers_

#     # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
#     labels = clstr_A.labels_
#     score_A = silhouette_score(reduced_X, labels, metric='euclidean')
#     scores_A[n] = score
    
#     print("Silhouette score with n = {}: {}".format(n, score))

# max_key_A = max(scores_A.keys(), key=(lambda k: scores_A[k])) 

# clstr_A = AgglomerativeClustering(n_clusters = max_key)
# clstr_A.fit(reduced_X)

# # TODO: Predict the cluster for each data point
# preds_A = clstr_A.predict(reduced_X)

# # TODO: Find the cluster centers
# centers_A = clstr_A.cluster_centers_

# # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
# labels_A = clstr_A.labels_
# score_A = silhouette_score(reduced_X, labels, metric='euclidean')
        
# print("Cluster centers: \n{}\n".format(centers_A))
# print("Silhouette score (max) with n = {}: {}".format(max_key, score_A))